# Popularity Recommendation - Group Merchants

The purpose here is to check whether need to divide merhcnats into groups based on the products they sell, so that during popularity recommendations, it can target some merchantes based on the main categories the store sells.

In [1]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
print(all_order_train.shape)

all_order_train.head()

(33720820, 12)


,order_id,user_id,order_number,order_hour_of_day,product_id,purchase_date,merchant,product_name,price,aisle,department,week_number
0,2539329,1,1,8,196,2019-02-13,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,7
1,2231262,31,17,11,196,2019-03-10,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
2,3058369,195,34,10,196,2019-02-25,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,9
3,2257155,951,3,20,196,2019-03-09,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
4,1121647,992,7,10,196,2019-03-08,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10


In [3]:
merchant_prod_cts = all_order_train[['merchant', 'product_id']].astype('str').drop_duplicates()\
                                                  .groupby(['merchant'], as_index=False)[['product_id']]\
                                                  .agg('count').reset_index(drop=True)
print(merchant_prod_cts.shape)

merchant_prod_cts.head()

(49, 2)


,merchant,product_id
0,49th Parallel Grocery,37655
1,Asian Food Centre,37841
2,Askew's Foods,37575
3,Avril (Health Supermarket),37675
4,Calgary Co-op,36093


In [4]:
print(min(merchant_prod_cts['product_id']), max(merchant_prod_cts['product_id']))

13796 37841


In [5]:
merchant_prod_cts = merchant_prod_cts.sort_values(by='product_id')
merchant_prod_cts.head()

,merchant,product_id
48,Vince's Market,13796
45,TaiKo Supermarket,14683
44,Sunterra Market,15236
39,Quality Foods,16188
43,Subzi Mandi Cash & Carry,16971


In [6]:
smallest_merchant = all_order_train.loc[all_order_train['merchant']=="Vince's Market"]
smallest_merchant['department'].value_counts()

produce            20420
dairy eggs         11674
snacks              6265
beverages           5808
frozen              4999
pantry              3898
bakery              2594
canned goods        2357
deli                2340
dry goods pasta     1844
household           1660
breakfast           1540
meat seafood        1423
personal care       1051
babies               810
international        589
alcohol              319
pets                 225
missing              163
other                 82
bulk                  70
Name: department, dtype: int64

In [7]:
product_values = pd.read_pickle('product_values.pkl')
product_values.head()

,product_id,total_popularity,product_value,avg_daily_sales
24517,24852,6935.67,31742.000,21.52
12984,13176,5385.83,12622.375,12.05
20846,21137,3708.33,6519.100,9.38
21602,21903,3537.39,15351.590,20.62
46574,47209,3003.98,6164.910,10.62


In [8]:
sample_products_set = smallest_merchant['product_id'].unique()
print(len(sample_products_set))

recommended_products = product_values.loc[~product_values['product_id'].isin(sample_products_set)]\
                                     .sort_values(by=['total_popularity', 'product_value'], 
                                                  ascending=[False, False]).head(n=10).reset_index(drop=True)
recommended_products

13796


,product_id,total_popularity,product_value,avg_daily_sales
0,43568,46.44,16.000,1.28
1,37658,44.86,306.180,17.01
2,34745,44.51,440.105,38.27
3,2075,42.99,247.355,8.11
4,37642,40.68,43.795,4.61
5,40940,39.70,192.240,7.12
6,34450,39.09,156.530,14.23
7,31623,38.32,43.470,2.07
8,19322,38.13,101.490,5.97
9,36543,37.95,261.630,9.69


## Summary

Even for the smallest mercahnt, it sells a wide variety of products, popularity recommendation won't raise its sales a lot in the following 1 week.